In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_json("/kaggle/input/two-sigma-connect-rental-listing-inquiries/train.json.zip")
test = pd.read_json("/kaggle/input/two-sigma-connect-rental-listing-inquiries/test.json.zip")

In [ ]:
display(train,test)

In [ ]:
# no = train[train["description"].apply(lambda x:x[:2]) == "No"]
# no["description"] = no["description"].apply(lambda x:x[:2])

# plt.figure(figsize=(20,12))
# sns.scatterplot(no["interest_level"],no["description"])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data["dinning_room"] = 0
all_data['laundry_in'] = 0
all_data['dog_allowed'] = 0


In [ ]:
sett = set()
for i in range(124011):
    sett.update(all_data["features"][i])

In [ ]:
sett

In [ ]:
list_index = []
dic = ["Dining Room","Laundry in Building","Dogs Allowed"]
for listnum,dic in zip(range(15,18),dic):
    for index,i in enumerate(all_data["features"]):
        for j in range(len(i)):
            if i[j] == dic:
                list_index.append(index) 
                print(i)
                break
    all_data.iloc[list_index,listnum] = 1 #이런 방식으로 데이터프레임에서 특정 위치의 값만 변경할 수 있다.
    list_index = []

In [ ]:
all_data

In [ ]:
all_data=all_data.drop(["photos","description","features","interest_level","building_id","manager_id"], axis=1)

In [ ]:
all_data["street_address"].value_counts()

In [ ]:
all_data["long_latitude"] = all_data["longitude"] + all_data["latitude"] #교호 작용


In [ ]:
all_data["created"] = pd.to_datetime(all_data["created"])
all_data.dtypes

In [ ]:
all_data["day"] = all_data["created"].dt.day
all_data["month"] = all_data["created"].dt.month
all_data["hour"] = all_data["created"].dt.hour
all_data["minute"] = all_data["created"].dt.minute
all_data["weekday"] = all_data["created"].dt.weekday

In [ ]:
all_data = all_data.drop("created",1)

In [ ]:
all_data[all_data["longitude"] == 0].count()

In [ ]:
all_data[all_data["latitude"] == 0].count()

In [ ]:
plt.figure(figsize=(20,12))
sns.scatterplot(train["interest_level"],all_data["latitude"][:len(train)])

In [ ]:
all_data.columns[all_data.dtypes == "object"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
#features 컬럼은 왜 안되나
le = LabelEncoder()
for i in all_data.columns[all_data.dtypes == "object"]:
    print(i)
    all_data[i] = le.fit_transform(list(all_data[i]))

In [ ]:
all_data.dtypes

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
all_data2=ss.fit_transform(all_data)
all_data2=pd.DataFrame(all_data2)

In [ ]:
train2 = all_data[:len(train)]
test2= all_data[len(train):]

In [ ]:
# train2["interest_level"] = train["interest_level"]
# plt.figure(figsize=(20,12))
# sns.countplot(x="dog_allowed",hue="interest_level",data=train2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(train2,train["interest_level"])
result = rf.predict_proba(test2)

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=1500)
cat.fit(train2,train["interest_level"])
result2 = cat.predict_proba(test2)

In [ ]:
pd.DataFrame(rf.feature_importances_,train2.columns)

In [ ]:
from sklearn.model_selection import cross_val_score

# np.sqrt(-cross_val_score(ridge,train2,train["SalePrice"],n_jobs=-1, cv=10, scoring="neg_mean_squared_error").mean())


print(-cross_val_score(rf,train2,train["interest_level"],n_jobs=-1, cv=10, scoring="neg_log_loss").mean())

In [ ]:
pd.DataFrame(result)

In [ ]:
sub = pd.read_csv("/kaggle/input/two-sigma-connect-rental-listing-inquiries/sample_submission.csv.zip")
result[:,1]

In [ ]:
sub = pd.read_csv("/kaggle/input/two-sigma-connect-rental-listing-inquiries/sample_submission.csv.zip")
sub["high"]= result[:,0] * 0.6 + result2[:,0] * 0.4
sub["low"]= result[:,1] * 0.6 + result2[:,1] * 0.4
sub["medium"]= result[:,2] * 0.6+ result2[:,2] * 0.4
sub.to_csv("sub_interest_level.csv",index=0)